In [2]:
# new code
import numpy as np
import xarray as xr

test = xr.open_dataset(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\GLDAS_NOAH025_M.A200911.020.nc4')

In [12]:
test

<xarray.Dataset>
Dimensions:                (time: 1, bnds: 2, lon: 1440, lat: 600)
Coordinates:
  * time                   (time) datetime64[ns] 2009-11-01
  * lon                    (lon) float32 -179.9 -179.6 -179.4 ... 179.6 179.9
  * lat                    (lat) float32 -59.88 -59.62 -59.38 ... 89.62 89.88
Dimensions without coordinates: bnds
Data variables: (12/37)
    time_bnds              (time, bnds) datetime64[ns] 2009-11-01 2009-12-01
    Swnet_tavg             (time, lat, lon) float32 ...
    Lwnet_tavg             (time, lat, lon) float32 ...
    Qle_tavg               (time, lat, lon) float32 ...
    Qh_tavg                (time, lat, lon) float32 ...
    Qg_tavg                (time, lat, lon) float32 ...
    ...                     ...
    Rainf_f_tavg           (time, lat, lon) float32 ...
    Tair_f_inst            (time, lat, lon) float32 ...
    Qair_f_inst            (time, lat, lon) float32 ...
    Psurf_f_inst           (time, lat, lon) float32 ...
    SWdown_f_tavg          (time, lat, lon) float32 ...
    LWdown_f_tavg          (time, lat, lon) float32 ...
Attributes: (12/19)
    CDI:                    Climate Data Interface version 1.9.8 (https://mpi...
    Conventions:            CF-1.6
    history:                created on date: 2019-10-11T19:50:23.507
    source:                 Noah_v3.6 forced with Princeton_V2.2
    institution:            NASA GSFC
    missing_value:          -9999.0
    ...                     ...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  -59.875
    SOUTH_WEST_CORNER_LON:  -179.875
    DX:                     0.25
    DY:                     0.25
    CDO:                    Climate Data Operators version 1.9.8 (https://mpi...

In [18]:
test.Qs_acc

<xarray.DataArray 'Qs_acc' (time: 1, lat: 600, lon: 1440)>
[864000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2009-11-01
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float32 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
Attributes:
    standard_name:  surface_runoff_amount
    long_name:      Storm surface runoff
    units:          kg m-2
    cell_methods:   time: mean
    vmin:           0.0
    vmax:           3.3886292

In [1]:
'''
Import required packages
'''

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

import numpy as np
import pandas as pd

from tensorflow import keras
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
from keras.models import load_model
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers

import matplotlib.pyplot as plt
import glob,os
import random
import os,shutil

In [2]:
# data preparation without lookback -- One to One prediction (o2o)
def data_prepare_o2o(dataset,scaler1, scaler2, shuffle):
    
    dataX, dataY = [], []
    scale_feature = scaler1.transform(dataset[:,:-1])
    scale_target = scaler2.transform(dataset[:,-1].reshape(-1, 1))
    for i in range(len(dataset)):
        feature = scale_feature[i]
        dataX.append(feature)
        target = scale_target[i]
        dataY.append(target)
        
    dataX, dataY = np.array(dataX), np.array(dataY)
    number_list = np.arange(0,len(dataX),1)
    random.shuffle(number_list) 
    dataX_new = []
    for index in number_list:
        dataX_new.append(dataX[index])
    dataY_new = []
    for index in number_list:
        dataY_new.append(dataY[index])

    if shuffle:
        return np.array(dataX_new), np.array(dataY_new)
    else:
        return np.array(dataX), np.array(dataY)

In [ ]:
# data preparation without lookback -- One to One prediction (o2o)
def data_prepare_static_o2o(dataset, static, scaler1, scaler2, scaler3, shuffle):
    
    dataX, dataY = [], []
    scale_feature = scaler1.transform(dataset[:,:-1])
    scale_target = scaler2.transform(dataset[:,-1].reshape(-1, 1))
    scale_static = scaler3.transform(static)
    for i in range(len(dataset)):
        feature = scale_feature[i]
        dataX.append(feature)
        target = scale_target[i]
        dataY.append(target)
        
    dataX, dataY = np.array(dataX), np.array(dataY)
    number_list = np.arange(0,len(dataX),1)
    random.shuffle(number_list) 
    dataX_new = []
    for index in number_list:
        dataX_new.append(dataX[index])
    dataY_new = []
    for index in number_list:
        dataY_new.append(dataY[index])

    if shuffle:
        return np.array(dataX_new), np.array(dataY_new)
    else:
        return np.array(dataX), np.array(dataY)

In [3]:
'''
Data load
'''

dataset = pd.read_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\data2_ML_new.csv',index_col = 0)

dynamic_feature_column = dataset.columns[8:18]
static_feature_column1 = dataset.columns[2:8]
static_feature_column2 = dataset.columns[19:]
target = dataset['NEE']

all_sites = dataset['Site'].unique()

result = pd.DataFrame(columns = all_sites, index = ['rmse_train', 'rmse_test', 'corr_train','corr_test'])

In [9]:
len(all_sites)

136

In [13]:
dataset_merged['classKoppen'].unique()

array(['BSk', 'Cfb', 'BWh', 'Aw', 'Cfa', 'BSh', 'Csa', 'Am', 'Dfb', 'Dfc',
       'Dwa', 'Dwb', 'ET', nan, 'Csb', 'Af', 'Dfd', 'Dfa', 'Dsb', 'Dsc',
       'Cwa'], dtype=object)

In [15]:
dataset_merged

,Site,VegType,lat,lon,timeStamp,Year,Month,ST,TD,TN,...,classKoppen,SWC_ERA,ST_ERA,PRE_ERA,soilType,high_vegType,low_vegType,high_vegCover,low_vegCover,Elavation
0,AR-SLu,MF,-33.4648,-66.4598,01-Jan-2009,2009,1,26.495,28.341,23.656,...,BSk,0.123921,301.780064,7.373480e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
1,AR-SLu,MF,-33.4648,-66.4598,01-Feb-2009,2009,2,26.836,28.767,24.304,...,BSk,0.110645,301.461437,5.233760e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
2,AR-SLu,MF,-33.4648,-66.4598,01-Mar-2009,2009,3,26.027,28.053,23.779,...,BSk,0.120296,299.640865,7.520206e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
3,AR-SLu,MF,-33.4648,-66.4598,01-Apr-2009,2009,4,23.475,25.733,21.352,...,BSk,0.084724,296.813533,4.464709e-06,4.0,19.0,16.0,0.104347,0.854505,503.525848
4,AR-SLu,MF,-33.4648,-66.4598,01-May-2009,2009,5,16.761,19.165,14.805,...,BSk,0.111681,289.083536,2.715185e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,GRA,38.0402,-121.7272,01-Mar-2020,2020,3,11.647,13.466,9.749,...,Csa,0.233008,286.323905,6.526713e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3783,US-Snf,GRA,38.0402,-121.7272,01-Apr-2020,2020,4,15.303,17.474,12.444,...,Csa,0.205334,290.409585,3.662211e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3784,US-Snf,GRA,38.0402,-121.7272,01-May-2020,2020,5,19.142,21.610,15.348,...,Csa,0.148441,294.903876,1.848216e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3785,US-Snf,GRA,38.0402,-121.7272,01-Jun-2020,2020,6,21.001,23.428,16.955,...,Csa,0.149445,297.000929,1.650937e-07,2.0,19.0,10.0,0.249163,0.729779,-2.341106


In [12]:
dataset_merged['VegType'].unique()

array(['MF', 'GRA', 'SAV', 'WSA', 'EBF', 'WET', 'DBF', 'CRO', 'ENF',
       'OSH', 'CSH', 'DNF', 'SNO'], dtype=object)

In [11]:
dataset_merged['soilType'].unique()

array([4., 2., 1., 3., 0., 6.])

In [18]:
'''
New Data load and merge
'''

dataset_new1 = pd.read_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\FluxNet_MM_new_Yikui.csv'
                           ,index_col = 0)

dataset_new2 = pd.read_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\data\AmeriFlux_MM_new_Yikui.csv'
                           ,index_col = 0)

frame = [dataset_new1.drop(columns = 'Unnamed: 0.1'), dataset_new2]
dataset_merged = pd.concat(frame)

all_sites = dataset_merged['Site'].unique()

result = pd.DataFrame(columns = all_sites, index = ['rmse_train', 'rmse_test', 'corr_train','corr_test'])

In [ ]:
#dynamic_feature_column = dataset.columns[8:18]
#static_feature_column1 = dataset.columns[2:8]
#static_feature_column2 = dataset.columns[19:]
#target = dataset['NEE']



In [6]:
len(all_sites)

171

In [7]:
dataset_merged

,Site,VegType,lat,lon,timeStamp,Year,Month,ST,TD,TN,...,classKoppen,SWC_ERA,ST_ERA,PRE_ERA,soilType,high_vegType,low_vegType,high_vegCover,low_vegCover,Elavation
0,AR-SLu,MF,-33.4648,-66.4598,01-Jan-2009,2009,1,26.495,28.341,23.656,...,BSk,0.123921,301.780064,7.373480e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
1,AR-SLu,MF,-33.4648,-66.4598,01-Feb-2009,2009,2,26.836,28.767,24.304,...,BSk,0.110645,301.461437,5.233760e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
2,AR-SLu,MF,-33.4648,-66.4598,01-Mar-2009,2009,3,26.027,28.053,23.779,...,BSk,0.120296,299.640865,7.520206e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
3,AR-SLu,MF,-33.4648,-66.4598,01-Apr-2009,2009,4,23.475,25.733,21.352,...,BSk,0.084724,296.813533,4.464709e-06,4.0,19.0,16.0,0.104347,0.854505,503.525848
4,AR-SLu,MF,-33.4648,-66.4598,01-May-2009,2009,5,16.761,19.165,14.805,...,BSk,0.111681,289.083536,2.715185e-05,4.0,19.0,16.0,0.104347,0.854505,503.525848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,GRA,38.0402,-121.7272,01-Mar-2020,2020,3,11.647,13.466,9.749,...,Csa,0.233008,286.323905,6.526713e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3783,US-Snf,GRA,38.0402,-121.7272,01-Apr-2020,2020,4,15.303,17.474,12.444,...,Csa,0.205334,290.409585,3.662211e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3784,US-Snf,GRA,38.0402,-121.7272,01-May-2020,2020,5,19.142,21.610,15.348,...,Csa,0.148441,294.903876,1.848216e-05,2.0,19.0,10.0,0.249163,0.729779,-2.341106
3785,US-Snf,GRA,38.0402,-121.7272,01-Jun-2020,2020,6,21.001,23.428,16.955,...,Csa,0.149445,297.000929,1.650937e-07,2.0,19.0,10.0,0.249163,0.729779,-2.341106


In [8]:
## categorical variables -- categorical features: Koppen, VegType, SoilType
onehot_merged_dataset = pd.get_dummies(dataset_merged, columns=['VegType', 'classKoppen', 'soilType'])

In [9]:
onehot_merged_dataset

,Site,lat,lon,timeStamp,Year,Month,ST,TD,TN,P,...,classKoppen_Dsc,classKoppen_Dwa,classKoppen_Dwb,classKoppen_ET,soilType_0.0,soilType_1.0,soilType_2.0,soilType_3.0,soilType_4.0,soilType_6.0
0,AR-SLu,-33.4648,-66.4598,01-Jan-2009,2009,1,26.495,28.341,23.656,1.698,...,0,0,0,0,0,0,0,0,1,0
1,AR-SLu,-33.4648,-66.4598,01-Feb-2009,2009,2,26.836,28.767,24.304,1.325,...,0,0,0,0,0,0,0,0,1,0
2,AR-SLu,-33.4648,-66.4598,01-Mar-2009,2009,3,26.027,28.053,23.779,2.650,...,0,0,0,0,0,0,0,0,1,0
3,AR-SLu,-33.4648,-66.4598,01-Apr-2009,2009,4,23.475,25.733,21.352,0.194,...,0,0,0,0,0,0,0,0,1,0
4,AR-SLu,-33.4648,-66.4598,01-May-2009,2009,5,16.761,19.165,14.805,0.220,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,38.0402,-121.7272,01-Mar-2020,2020,3,11.647,13.466,9.749,1.187,...,0,0,0,0,0,0,1,0,0,0
3783,US-Snf,38.0402,-121.7272,01-Apr-2020,2020,4,15.303,17.474,12.444,0.573,...,0,0,0,0,0,0,1,0,0,0
3784,US-Snf,38.0402,-121.7272,01-May-2020,2020,5,19.142,21.610,15.348,0.359,...,0,0,0,0,0,0,1,0,0,0
3785,US-Snf,38.0402,-121.7272,01-Jun-2020,2020,6,21.001,23.428,16.955,0.000,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
## lon/lat


In [ ]:
# static numerical features




In [19]:
# dynamic features and static features #

dynamic_dataset = dataset_merged.drop(columns=[ 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
       'Month', 'classKoppen', 'soilType', 'high_vegType', 'low_vegType',
       'high_vegCover', 'low_vegCover', 'Elavation'])

static_dataset = dataset_merged.drop(columns=list(dynamic_dataset.columns.drop(['Site'])))
static_dataset = static_dataset.drop(columns=['timeStamp','Year','high_vegType', 'low_vegType'])

In [24]:
dynamic_dataset

,Site,ST,TD,TN,P,H,LE,VPD,WS,SW,LW,SWC_ERA,ST_ERA,PRE_ERA,NEE
0,AR-SLu,26.495,28.341,23.656,1.698,113.22300,53.2518,17.691,2.273,311.714,362.361,0.123921,301.780064,7.370000e-05,-5.998810
1,AR-SLu,26.836,28.767,24.304,1.325,119.36300,55.7527,17.641,2.552,291.650,358.598,0.110645,301.461437,5.230000e-05,-5.995460
2,AR-SLu,26.027,28.053,23.779,2.650,129.29600,61.0429,15.800,2.323,239.598,358.482,0.120296,299.640865,7.520000e-05,-5.998610
3,AR-SLu,23.475,25.733,21.352,0.194,134.38200,63.1503,13.678,2.444,193.424,340.013,0.084724,296.813533,4.460000e-06,-5.997700
4,AR-SLu,16.761,19.165,14.805,0.220,137.69500,64.7614,10.150,2.290,145.140,305.258,0.111681,289.083536,2.720000e-05,-5.998810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,11.647,13.466,9.749,1.187,15.04130,73.6048,5.589,2.924,193.323,304.937,0.233008,286.323905,6.530000e-05,-2.799720
3783,US-Snf,15.303,17.474,12.444,0.573,9.28612,118.1150,8.360,3.699,268.291,314.860,0.205334,290.409585,3.660000e-05,-0.859376
3784,US-Snf,19.142,21.610,15.348,0.359,43.47910,93.5528,11.840,4.050,313.248,324.195,0.148441,294.903876,1.850000e-05,4.552360
3785,US-Snf,21.001,23.428,16.955,0.000,38.14970,112.7220,13.158,4.732,360.256,328.495,0.149445,297.000929,1.650000e-07,3.041250


In [11]:
static_dataset

,Site,VegType,lat,lon,Month,classKoppen,soilType,high_vegCover,low_vegCover,Elavation
0,AR-SLu,MF,-33.4648,-66.4598,1,BSk,4.0,0.104347,0.854505,503.525848
1,AR-SLu,MF,-33.4648,-66.4598,2,BSk,4.0,0.104347,0.854505,503.525848
2,AR-SLu,MF,-33.4648,-66.4598,3,BSk,4.0,0.104347,0.854505,503.525848
3,AR-SLu,MF,-33.4648,-66.4598,4,BSk,4.0,0.104347,0.854505,503.525848
4,AR-SLu,MF,-33.4648,-66.4598,5,BSk,4.0,0.104347,0.854505,503.525848
...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,GRA,38.0402,-121.7272,3,Csa,2.0,0.249163,0.729779,-2.341106
3783,US-Snf,GRA,38.0402,-121.7272,4,Csa,2.0,0.249163,0.729779,-2.341106
3784,US-Snf,GRA,38.0402,-121.7272,5,Csa,2.0,0.249163,0.729779,-2.341106
3785,US-Snf,GRA,38.0402,-121.7272,6,Csa,2.0,0.249163,0.729779,-2.341106


In [21]:

"""
Data engineering -- encode month; static variables -- class for encoding

"""
# month
# cos, sin
import numpy as np

static_dataset['month_sin'] = np.sin(static_dataset.Month*(2.*np.pi/12))
static_dataset['month_cos'] = np.cos(static_dataset.Month*(2.*np.pi/12))


In [14]:
static_dataset

,Site,VegType,lat,lon,Month,classKoppen,soilType,high_vegCover,low_vegCover,Elavation,month_sin,month_cos
0,AR-SLu,MF,-33.4648,-66.4598,1,BSk,4.0,0.104347,0.854505,503.525848,5.000000e-01,8.660254e-01
1,AR-SLu,MF,-33.4648,-66.4598,2,BSk,4.0,0.104347,0.854505,503.525848,8.660254e-01,5.000000e-01
2,AR-SLu,MF,-33.4648,-66.4598,3,BSk,4.0,0.104347,0.854505,503.525848,1.000000e+00,6.123234e-17
3,AR-SLu,MF,-33.4648,-66.4598,4,BSk,4.0,0.104347,0.854505,503.525848,8.660254e-01,-5.000000e-01
4,AR-SLu,MF,-33.4648,-66.4598,5,BSk,4.0,0.104347,0.854505,503.525848,5.000000e-01,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,GRA,38.0402,-121.7272,3,Csa,2.0,0.249163,0.729779,-2.341106,1.000000e+00,6.123234e-17
3783,US-Snf,GRA,38.0402,-121.7272,4,Csa,2.0,0.249163,0.729779,-2.341106,8.660254e-01,-5.000000e-01
3784,US-Snf,GRA,38.0402,-121.7272,5,Csa,2.0,0.249163,0.729779,-2.341106,5.000000e-01,-8.660254e-01
3785,US-Snf,GRA,38.0402,-121.7272,6,Csa,2.0,0.249163,0.729779,-2.341106,1.224647e-16,-1.000000e+00


In [ ]:
# skip now
# group into season

def season(df, metadata):
    """Add season column based on lat and month
    """
    site = df['site'].cat.categories.item()
    lat = metadata[metadata['site'] == site]['lat'].item()
    if lat > 0:
        seasons = {3: 'spring',  4: 'spring',  5: 'spring',
                   6: 'summer',  7: 'summer',  8: 'summer',
                   9: 'fall',   10: 'fall',   11: 'fall',
                  12: 'winter',  1: 'winter',  2: 'winter'}
    else:
        seasons = {3: 'fall',    4: 'fall',    5: 'fall',
                   6: 'winter',  7: 'winter',  8: 'winter',
                   9: 'spring', 10: 'spring', 11: 'spring',
                  12: 'summer',  1: 'summer',  2: 'summer'}
    return df.assign(season=df.TIMESTAMP.dt.month.map(seasons))


data = pd.get_dummies(data, columns=['season']).assign(season=data['season'])

In [22]:
## categorical variables -- categorical features: Koppen, VegType, SoilType


onehot_static_dataset = pd.get_dummies(static_dataset, columns=['VegType', 'classKoppen', 'soilType'])

In [26]:
onehot_static_dataset.drop(columns=['Month'])

,Site,lat,lon,high_vegCover,low_vegCover,Elavation,month_sin,month_cos,VegType_CRO,VegType_CSH,...,classKoppen_Dsc,classKoppen_Dwa,classKoppen_Dwb,classKoppen_ET,soilType_0,soilType_1,soilType_2,soilType_3,soilType_4,soilType_6
0,AR-SLu,-33.4648,-66.4598,0.104347,0.854505,503.525848,5.000000e-01,8.660254e-01,0,0,...,0,0,0,0,0,0,0,0,1,0
1,AR-SLu,-33.4648,-66.4598,0.104347,0.854505,503.525848,8.660254e-01,5.000000e-01,0,0,...,0,0,0,0,0,0,0,0,1,0
2,AR-SLu,-33.4648,-66.4598,0.104347,0.854505,503.525848,1.000000e+00,6.123234e-17,0,0,...,0,0,0,0,0,0,0,0,1,0
3,AR-SLu,-33.4648,-66.4598,0.104347,0.854505,503.525848,8.660254e-01,-5.000000e-01,0,0,...,0,0,0,0,0,0,0,0,1,0
4,AR-SLu,-33.4648,-66.4598,0.104347,0.854505,503.525848,5.000000e-01,-8.660254e-01,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3782,US-Snf,38.0402,-121.7272,0.249163,0.729779,-2.341106,1.000000e+00,6.123234e-17,0,0,...,0,0,0,0,0,0,1,0,0,0
3783,US-Snf,38.0402,-121.7272,0.249163,0.729779,-2.341106,8.660254e-01,-5.000000e-01,0,0,...,0,0,0,0,0,0,1,0,0,0
3784,US-Snf,38.0402,-121.7272,0.249163,0.729779,-2.341106,5.000000e-01,-8.660254e-01,0,0,...,0,0,0,0,0,0,1,0,0,0
3785,US-Snf,38.0402,-121.7272,0.249163,0.729779,-2.341106,1.224647e-16,-1.000000e+00,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
## lon/lat


In [ ]:
# summary of static variables

In [28]:
#site_dataset = dataset.loc[dataset['Site'] == all_sites[0]]

#dynamic_feature = site_dataset[dynamic_feature_column]
#static_feature1 = site_dataset[static_feature_column1]
#static_feature2 = site_dataset[static_feature_column2]


# split into train and test sets
train_size = int(len(dynamic_dataset.values) * 0.75)
test_size = len(dynamic_dataset.values) - train_size
train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]

# first split train/test and then apply scaler
scaler_x = StandardScaler()
scaler_x.fit(train[:,:-1])

scaler_y = StandardScaler()
scaler_y.fit(train[:,-1].reshape(-1, 1))

# train and test dataset
trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)

ValueError: could not convert string to float: 'AR-SLu'

In [10]:
trainY.shape

(42, 1)

In [11]:
### EXP1 ###

from sklearn.preprocessing import StandardScaler
import math
from scipy.stats import pearsonr

for i in range(len(all_sites)):
    site_dataset = dataset.loc[dataset['Site'] == all_sites[i]]

    #dynamic_feature = site_dataset[dynamic_feature_column]
    #static_feature1 = site_dataset[static_feature_column1]
    #static_feature2 = site_dataset[static_feature_column2]

    dynamic_dataset = site_dataset.drop(columns=['Unnamed: 0.1', 'Site', 'VegType', 'lat', 'lon', 'timeStamp', 'Year',
           'Month', 'classKoppen', 'SWC', 'soilType', 'high_vegType', 'low_vegType',
           'high_vegCover', 'low_vegCover', 'Elavation'])


    # split into train and test sets
    train_size = int(len(dynamic_dataset.values) * 0.75)
    test_size = len(dynamic_dataset.values) - train_size
    train, test = dynamic_dataset.values[0:train_size,:], dynamic_dataset.values[train_size:len(dynamic_dataset.values),:]
    
    # first split train/test and then apply scaler
    scaler_x = StandardScaler()
    scaler_x.fit(train[:,:-1])

    scaler_y = StandardScaler()
    scaler_y.fit(train[:,-1].reshape(-1, 1))
    
    # train and test dataset
    trainX, trainY = data_prepare_o2o(train, scaler_x, scaler_y, True)
    testX, testY = data_prepare_o2o(test, scaler_x, scaler_y, True)
    
    '''
    Valina LSTM
    '''
    # define customer optimizer
    #RMSprop= optimizers.RMSprop(lr=0.0001)
    # define special callbacks
    #reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    # define early stop
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0)

    model = Sequential()
    model.add(LSTM(32, input_shape=(trainX.shape[1], 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=300, validation_split=0.1, batch_size=5, callbacks=[early_stopping], verbose=0)
    
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY = scaler_y.inverse_transform(trainY)
    testPredict = scaler_y.inverse_transform(testPredict)
    testY = scaler_y.inverse_transform(testY)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY.squeeze(), trainPredict[:,0]))
    #print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY.squeeze(), testPredict[:,0]))
    #print('Test Score: %.2f RMSE' % (testScore))
    # calculate correlation score
    corr_train, _ = pearsonr(trainY.squeeze(), trainPredict[:,0])
    corr_test, _ = pearsonr(testY.squeeze(), testPredict[:,0])
    
    result[all_sites[i]] = [trainScore, testScore, corr_train, corr_test]
    
    print (all_sites[i])
    

AR-SLu
AT-Neu
AU-ASM
AU-Ade
AU-Cpr
AU-Cum
AU-DaP
AU-DaS
AU-Dry
AU-Emr
AU-Fog
AU-GWW
AU-Gin
AU-How
AU-Lox
AU-RDF
AU-Rig
AU-Rob
AU-Stp
AU-TTE
AU-Tum
AU-Wac
AU-Whr
AU-Wom
AU-Ync
BE-Bra
BE-Lon
BE-Vie
BR-Sa3
CA-Gro
CA-Obs
CA-Qfo
CA-SF1
CA-SF2
CA-SF3
CA-TP1
CA-TP2
CA-TP3
CA-TP4
CA-TPD
CH-Cha
CH-Dav
CH-Fru
CN-Cng
CN-Du2
CN-Du3
CN-HaM
CZ-wet
DE-Geb
DE-Gri
DE-Hai
DE-Kli
DE-Lkb
DE-Lnf
DE-Obe
DE-RuR
DE-RuS
DE-Seh
DE-SfN
DE-Tha
DE-Zrk
DK-Eng
DK-Fou
DK-Sor
ES-Amo
ES-LJu
ES-LgS
FI-Hyy
FI-Jok
FI-Let
FI-Lom
FI-Sod
FR-Gri
FR-LBr
FR-Pue
GH-Ank
GL-ZaF
IT-BCi
IT-CA1
IT-CA2
IT-CA3
IT-Col
IT-Cp2
IT-Cpz
IT-Isp
IT-Lav
IT-MBo
IT-Noe
IT-PT1
IT-Ren
IT-Ro2
IT-SR2
IT-SRo
IT-Tor
MY-PSO
NL-Hor
NL-Loo
RU-Cok
RU-Fyo
RU-Ha1
RU-SkP
RU-Tks
RU-Vrk
SD-Dem
SJ-Adv
SN-Dhr
US-AR1
US-AR2
US-ARM
US-ARb
US-ARc
US-Atq
US-Blo
US-CRT
US-Cop
US-GBT
US-GLE
US-Goo
US-IB2
US-Ivo
US-KS1
US-KS2
US-LWW
US-Lin
US-Los
US-MMS
US-Me1
US-Me2
US-Me4
US-Me5
US-NR1
US-Ne1
US-Ne2
US-Ne3
US-Oho
US-Prr


In [12]:
result.to_csv(r'C:\Users\Macbook Air\Desktop\NEE_forecast\result\result_exp1_new.csv')

In [ ]:
# visualization #